In [1]:
import pandas as pd
#import pymysql
import numpy as np

In [2]:
    
# coding: utf-8

# In[1]:




# mysql_cn= pymysql.connect(host='localhost', 
#                 port=3306,user='root', passwd='Bey_88888', 
#                 db='Data_mining_Sachin')


# In[2]:




question_and_tags= pd.read_csv('question_and_tags.csv')

# In[3]:


tags = pd.read_csv("tags.csv")


# In[4]:


tagCol = tags.loc[:,['TagName']]

tagIdCol = tags.loc[:,['Id']]


tagList=tagCol.TagName.unique()


# In[5]:


tagCount = tagList.size  #tagCount contain total # of tag count


# In[6]:


# print(tagCount)


# In[7]:


tagsFromQuestion = question_and_tags.loc[:,['tags']]
questionList = question_and_tags.loc[:,['Id']]


# In[8]:


# dictionary for question tag
tagDictionary = {}
index=0

In [3]:
for index,row in tags.iterrows():
    
#     print("row ",row)
    
    tagDictionary[row.TagName]=index
    index+=1

In [4]:




# In[9]:


# print(tagDictionary)


# In[10]:


# code for computing co-ocurrence of tags in tagMatrix
tagDimension = (tagCount,tagCount)

tagMatrix = np.zeros(tagDimension)

import re

for index,row in question_and_tags.iterrows():
    questionTag = row.tags
    #tempTagList = []
    regex="<*>?"
    match=re.split(regex,questionTag)   
    #print(type(match))
    #print("Question ID ==> ",row.Id," "," : Tags ==> ",match)
    for temp_str in match:
        if(len(temp_str)!=0):
            for temp_tag in match:
                if(len(temp_tag)!=0):
                    #print("TempTag ==> ",temp_tag," temp_str ==> ",temp_str)
                    if((tagDictionary.get(temp_tag) is not None) and (tagDictionary.get(temp_str) is not None) ):
                        tagMatrix[tagDictionary[temp_str]][tagDictionary[temp_tag]]+=1
                        
                    
                


# In[11]:


# computing cosine similarity between tags, we get cosineMatrix
import math
tagDimension = (tagCount,tagCount)

cosineMatrix = np.zeros(tagDimension)

for i in range(tagCount):
    for j in range(tagCount):
        tempNum = tagMatrix[i][j]
        tempDeno = tagMatrix[i][i]*tagMatrix[j][j]
        if(tempDeno!=0):
            cosineMatrix[i][j]=tempNum/math.sqrt(tempDeno)
            
        
        


# In[12]:


# print((cosineMatrix[0]))


# In[13]:


import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
X=np.array(cosineMatrix)
from sklearn.cluster import KMeans
clf=KMeans(n_clusters=20)
clf.fit(X)
centroids=clf.cluster_centers_
labels=clf.labels_
colors= 10*["g.","r.","c.","b.","k."]
for i in range(len(X)):
    plt.plot(X[i][0],X[i][1],colors[labels[i]],markersize=25)
plt.scatter(centroids[:,0],centroids[:,1],marker='x',s=150,linewidth=5)
plt.show()
# spectral = SpectralClustering(n_clusters=10, eigen_solver='arpack', affinity='precomputed',assign_labels='discretize')
# labels = spectral_clustering(X ,n_clusters=50)



# In[14]:


# print(labels)


# In[15]:


clusterSize = np.unique(labels).size

# print("Cluster size ==> ",clusterSize)


# In[16]:


# for mapping every tag with its respective topic/cluster, we get tagClusterDict
tagClusterDict = {}
for t,l in zip(tagList,labels):
    tagClusterDict[t]=l
    


# In[17]:


# print(tagClusterDict)


# In[18]:


import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))


# In[19]:


# clusterDict to store number of tags in each cluster
from collections import defaultdict
clusterDict=defaultdict(int)
for t in labels:
    clusterDict[t]+=1


# In[20]:


# print(clusterDict)


# In[21]:


userDict = {}


# In[22]:


question_set = pd.read_csv("question_set_query.csv")
# print(type(question_set))


# In[23]:


question_set = question_set[question_set.user_id!=0]

question_set = question_set[question_set.user_id!=-1]


# In[24]:


# print(question_set.size)
userGroup = question_set.groupby('user_id')


# In[25]:


userKeys = userGroup.groups.keys()


# In[26]:


# print(type(userKeys))


# In[27]:


import re


# In[28]:


allTags =pd.read_csv("allTags.csv")

allScores= pd.read_csv("allScores.csv")


topic_exp={}

from collections import defaultdict

#tagClusterDict contains the tag to cluster ID mapping
#clusterDict contains the total count of tags against each and every cluster

for user in userKeys:
    temp_q_list = userGroup.get_group(user).values[:,0:1]
    #sum=0
    tempClusDict=defaultdict(int)
    #Loop for question
    for temp_y in temp_q_list:
        tagset=set()
        
        
        #result = pd.read_sql("select Tags from posts where Id=%d;"%temp_y[0],mysql_cn)
        
        result=allTags.loc[allTags['Id']==temp_y[0]]
        #print(result)
        
        tags=result.Tags
        tags=tags.values[0]
        
#         print("Tags ==> ",tags)
        regex="<*>?"
        match=re.split(regex,tags)        
        for temp_str in match:
            if(len(temp_str)!=0):
                tagset.add(temp_str)
        
        #result=pd.read_sql("select Score from posts where Id=%d;"%temp_y[0],mysql_cn)
        result=allScores.loc[allScores['Id']==temp_y[0]]
        score = result.Score
        score = score.values[0]
#         print("Score ==> ",score)
        
        for tempTag in tagset:
            if(tagClusterDict.get(tempTag) is not None):
                clusterId=tagClusterDict[tempTag]
                totalTagCount = clusterDict[clusterId]
                tempClusDict[clusterId]+=((1/totalTagCount)*sigmoid(score))
            
        
        
       
    
    topic_exp[user]=tempClusDict
        
    tempClusDict=None


# In[29]:


# print(topic_exp[5])


# In[30]:




# In[31]:

#total_ans_user_query.csv

total_ans_df = pd.read_csv("total_ans_user_query.csv")


# In[32]:




# In[33]:


total_ans_df.head(2)

total_ans_dict = total_ans_df.set_index('user_id').T.to_dict()

#print(total_ans_dict[2]['answer_count'])


# In[34]:

#total_ans_user_by_hour_query.csv

#total_ans_by_hour_df=pd.read_sql(total_ans_user_by_hour_query,con=mysql_cn)
total_ans_by_hour_df=pd.read_csv("total_ans_user_by_hour_query.csv")

# In[35]:


total_ans_by_hour_df.head(2)


# In[36]:


from collections import defaultdict

hourDict=defaultdict(dict)


# In[37]:


for index,row in total_ans_by_hour_df.iterrows():
    hour = row["hour_of_ans"]
    user = row["user_id"]
    ans_count = row["answer_count"]
    
    if(hourDict.get(hour) is None):
        userDict = defaultdict(int)
        
    else:
        userDict=hourDict[hour]
    
    totalAnsCount = total_ans_dict[user]['answer_count']
    userDict[user]+=(ans_count/totalAnsCount)
    hourDict[hour]=userDict


# In[38]:


# print(hourDict)


# In[39]:


import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().magic('matplotlib inline')


# In[40]:


# #set font size of labels on matplotlib plots
# plt.rc('font', size=16)

# #set style of plots
# sns.set_style('white')

# #define a custom palette
# customPalette = ['#630C3A', '#39C8C6', '#D3500C', '#FFB139']
# sns.set_palette(customPalette)
# sns.palplot(customPalette)


# In[41]:


# import time
# import pylab as pl
# plot_num = 1
# for algorithm in [two_means, labels]:
#         # predict cluster memberships
#         t0 = time.time()
#         algorithm.fit(X)
#         t1 = time.time()
#         if hasattr(algorithm, 'labels_'):
#             y_pred = algorithm.labels_.astype(np.int)
#         else:
#             y_pred = algorithm.predict(X)

#         # plot
#         pl.subplot(1, 2, plot_num)
#             #if i_dataset == 0:
#             #pl.title(str(algorithm).split('(')[0], size=18)
#         pl.scatter(X[:, 0], X[:, 1], color=colors[y_pred].tolist(), s=10)

#         if hasattr(algorithm, 'cluster_centers_'):
#             centers = algorithm.cluster_centers_
#             center_colors = colors[:len(centers)]
#             pl.scatter(centers[:, 0], centers[:, 1], s=100, c=center_colors)
#         pl.xlim(-2.5, 2.5)
#         pl.ylim(-2.5, 2.5)
#         pl.xticks(())
#         pl.yticks(())
#         pl.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
#                 transform=pl.gca().transAxes, size=15,
#                 horizontalalignment='right')
#         plot_num += 1

# pl.show()


# In[42]:


# # Show
# plt.scatter(X[:, 0], X[:, 1],
#             c=labels, edgecolor='')



# In[43]:


# plt.xlabel('PC1')
# plt.ylabel('PC2')
# plt.show()





/opt/anaconda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


/opt/anaconda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [5]:
# In[82]:


#tagQuery="select  hour(pst.CreationDate) hour_of_ans  , pst.Tags tags from posts pst  , post_history pst_hist  where pst.PostTypeId=1 and pst.Id=pst_hist.PostId  and pst_hist.UserId NOT IN (0,-1)  group by hour(pst.CreationDate) , pst.Tags order by hour(pst.CreationDate) asc;"


# In[83]:

#tagQuery.csv
tagHour = pd.read_csv("tagQuery.csv")



In [6]:
tagHour.head(4)



,hour_of_ans,tags
0,0,<.net><asp.net><asp.net-mvc><webforms>
1,0,<.net><continuous-integration><builds><node.js...
2,0,<.net><generics>
3,0,<.net><html><css><information>


In [7]:
tagHour1=tagHour.copy()


In [8]:
userTempList=[]
        
for user in userKeys:
    t="Avail_"+str(user)
    userTempList.append(t)
    

tagHour1=pd.concat([tagHour1,pd.DataFrame(columns=userTempList)],axis=1)





In [9]:
clusterList=[]



for lbl in range(clusterSize):
    t="Clust_"+str(lbl)
#     print(t)
    clusterList.append(t)

tagHour1=pd.concat([tagHour1,pd.DataFrame(columns=clusterList)],axis=1)    

In [10]:
tagHour1.head(2)

,hour_of_ans,tags,Avail_3,Avail_4,Avail_5,Avail_6,Avail_7,Avail_9,Avail_11,Avail_12,...,Clust_10,Clust_11,Clust_12,Clust_13,Clust_14,Clust_15,Clust_16,Clust_17,Clust_18,Clust_19
0,0,<.net><asp.net><asp.net-mvc><webforms>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,<.net><continuous-integration><builds><node.js...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
userExpTagList=[]
        
for user in userKeys:
    t="UserExpTagForUserID_"+str(user)
    userExpTagList.append(t)
    

tagHour1=pd.concat([tagHour1,pd.DataFrame(columns=userExpTagList)],axis=1)






In [13]:
tagHour1.head(2)

,hour_of_ans,tags,Avail_3,Avail_4,Avail_5,Avail_6,Avail_7,Avail_9,Avail_11,Avail_12,...,UserExpTagForUserID_289358,UserExpTagForUserID_289431,UserExpTagForUserID_289442,UserExpTagForUserID_289522,UserExpTagForUserID_289552,UserExpTagForUserID_289679,UserExpTagForUserID_289738,UserExpTagForUserID_289765,UserExpTagForUserID_289779,UserExpTagForUserID_289855
0,0,<.net><asp.net><asp.net-mvc><webforms>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,<.net><continuous-integration><builds><node.js...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#topic_exp
mostTopicExp={}
for tempUser in userTempList:
    t1=int(tempUser.split("_")[1])
    exp=topic_exp[t1]
    maxScore=-999999999
    maxClustId=-9999999
    for tK in exp.keys():
        if(exp[tK]>maxScore):
            maxScore=exp[tK]
            maxClustId=tK
    
#     print("For UserID ",t1," , most expertise in cluster ID ",maxClustId)
    mostTopicExp[t1]=maxClustId




In [15]:
type(tagHour1)

pandas.core.frame.DataFrame

In [16]:
# tagHour2=tagHour1.copy()

In [17]:
# tagHour2.head(1)

In [18]:
aa=pd.DataFrame([[1,2,3,4],[7,6,5,4],[1,9,4,6]])

In [19]:
aa

,0,1,2,3
0,1,2,3,4
1,7,6,5,4
2,1,9,4,6


In [20]:
# for index,row in aa.iterrows():
#     print(index,row[1],row[3])
#     if(index==1):
#         break

In [ ]:
for index,row in tagHour1.iterrows():
    hour=row['hour_of_ans']
    tags=row['tags']
    regex="<*>?"
    tempTagSet=set()
    for temp_array in tags:
        match=re.split(regex,(temp_array))
        for temp_str in match:
            if(len(temp_str)!=0):
                tempTagSet.add(temp_str)
                
    tempList=(list(tempTagSet))  
    tagHour1.at[index,'hour_of_ans']=hour
    tagHour1.at[index,'tags']=tempList
    
    for tempUser in userTempList:
        t1=int(tempUser.split("_")[1])
        tagHour1.at[index,tempUser]=hourDict[hour][t1]
        
    for tags in tempList:
        if(tagClusterDict.get(tags)is not None):
            clustId=tagClusterDict[tags]
            tagHour1.at[index,"Clust_"+str(clustId)]=1
          
    for userExpertiseTemp in userExpTagList:
        t1=int(userExpertiseTemp.split("_")[1])
        tagHour1.at[index,userExpertiseTemp]=mostTopicExp[t1]
            
      
                
                
            
    
    
    
    
    
    
    

/opt/anaconda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [ ]:
aa[2]

In [ ]:
a=str(tagHour1['tags'].head(1))

In [ ]:
a

In [ ]:
for i in range(len(a)):
    if a[i]=='<':
        z=''
        for j in range(i+1,len(a)):
            if(a[j]!= '>'):
                z=z+a[j]
            else:
                i=j
                print(z)
                break

In [ ]:
tagDictionary['.net']

In [ ]:
import re

for index,row in tagHour1.iterrows():
    print("Index ==> ",index," ","Row ==> ",row)
#     temp_tag_list = hourGroup.get_group(hour).values[:,1]
#     tempSet=set()
#     if(temp_tag_list.size>0):
        
#         regex="<*>?"
#         for temp_array in temp_tag_list:
#             match=re.split(regex,(temp_array))
#             for temp_str in match:
#                 if(len(temp_str)!=0):
#                     tempSet.add(temp_str)
                    
                    
#     tempList=(list(tempSet))  
#     tagTempHour.at[index,'hour_of_ans']=hour
#     tagTempHour.at[index,'tags']=tempList
    
#     for tempUser in userTempList:
#         t1=int(tempUser.split("_")[1])
#         tagTempHour.at[index,tempUser]=hourDict[hour][t1]
        
    
#     for tags in tempList:
#         clustId=tagClusterDict[tags]
#         tagTempHour.at[index,"Clust_"+str(clustId)]=1
        
        
#     for userExpertiseTemp in userExpTagList:
#         t1=int(userExpertiseTemp.split("_")[1])
#         tagTempHour.at[index,userExpertiseTemp]=mostTopicExp[t1]
        
        
    
#     index+=1

In [ ]:
hourGroup = tagHour.groupby('hour_of_ans')

hourKeys = hourGroup.groups.keys()

In [ ]:
from ast import literal_eval

In [ ]:
tagTempHour=pd.DataFrame(columns=['hour_of_ans','tags'])

In [ ]:
tagTempHour.head(2)

In [ ]:
userTempList=[]
        
for user in userKeys:
    t="Avail_"+str(user)
    userTempList.append(t)
    

tagTempHour=pd.concat([tagTempHour,pd.DataFrame(columns=userTempList)],axis=1)




In [ ]:
tagTempHour.head(2)

In [ ]:
clusterList=[]



for lbl in range(clusterSize):
    t="Clust_"+str(lbl)
#     print(t)
    clusterList.append(t)

tagTempHour=pd.concat([tagTempHour,pd.DataFrame(columns=clusterList)],axis=1)    

In [ ]:
tagTempHour.head(2)

In [ ]:
userExpTagList=[]
        
for user in userKeys:
    t="UserExpTagForUserID_"+str(user)
    userExpTagList.append(t)
    

tagTempHour=pd.concat([tagTempHour,pd.DataFrame(columns=userExpTagList)],axis=1)





In [ ]:
tagTempHour.head(2)

In [ ]:
#topic_exp
mostTopicExp={}
for tempUser in userTempList:
    t1=int(tempUser.split("_")[1])
    exp=topic_exp[t1]
    maxScore=-999999999
    maxClustId=-9999999
    for tK in exp.keys():
        if(exp[tK]>maxScore):
            maxScore=exp[tK]
            maxClustId=tK
    
    print("For UserID ",t1," , most expertise in cluster ID ",maxClustId)
    mostTopicExp[t1]=maxClustId



In [ ]:
import re
index=0
for hour in hourKeys:
    temp_tag_list = hourGroup.get_group(hour).values[:,1]
    tempSet=set()
    if(temp_tag_list.size>0):
        
        regex="<*>?"
        for temp_array in temp_tag_list:
            match=re.split(regex,(temp_array))
            for temp_str in match:
                if(len(temp_str)!=0):
                    tempSet.add(temp_str)
                    
                    
    tempList=(list(tempSet))  
    tagTempHour.at[index,'hour_of_ans']=hour
    tagTempHour.at[index,'tags']=tempList
    
    for tempUser in userTempList:
        t1=int(tempUser.split("_")[1])
        tagTempHour.at[index,tempUser]=hourDict[hour][t1]
        
    
    for tags in tempList:
        clustId=tagClusterDict[tags]
        tagTempHour.at[index,"Clust_"+str(clustId)]=1
        
        
    for userExpertiseTemp in userExpTagList:
        t1=int(userExpertiseTemp.split("_")[1])
        tagTempHour.at[index,userExpertiseTemp]=mostTopicExp[t1]
        
        
    
    index+=1

In [ ]:
tagTempHour.head(4)

In [ ]:
output=None

In [ ]:
userTempList=[]
        
for user in userKeys:
    t="Output_"+str(user)
    userTempList.append(t)
    
    
output=pd.concat([output,pd.DataFrame(columns=userTempList)],axis=1)

    

In [ ]:
output

In [ ]:
index=0
for hour in hourKeys:
    print("For Hour ===> ",hour)
    t=tagTempHour.loc[tagTempHour['hour_of_ans']==hour]
    t=t.filter(regex="Clust_")
    nonZeroCluster=[]
    for i in t:
        if(np.isnan(t[i].values[0])==False):
            clusterID=int(i.split("_")[1])
            nonZeroCluster.append(clusterID)
            
        
    tUserExp=tagTempHour.loc[tagTempHour['hour_of_ans']==hour]
    tUserExp=tUserExp.filter(regex="UserExpTagForUserID_")

    userExpList=[]

    for i in tUserExp:
        if(np.isnan(tUserExp[i].values[0])==False):
            if(tUserExp[i].values[0] in nonZeroCluster):
                userExpList.append(int(i.split("_")[1]))
    
    t=tagTempHour.loc[tagTempHour['hour_of_ans']==hour]
    #maxAvail=-999999
    #maxAvailUserId=-999999
    
    
    for temp in userExpList:
        avail=t["Avail_"+str(temp)].values[0]
        #print("Avail_"+str(temp)+" "+" ",avail)
        if(np.isnan(avail) or avail==0):
            output.at[index,'Output_'+str(temp)]=0
        else:
            output.at[index,'Output_'+str(temp)]=1
            
                
    index+=1
        
        
    
        
                


    

In [ ]:
output

In [ ]:
tagTempHour

In [ ]:
tagHour = tagTempHour.copy()

In [ ]:
## Removing tags column

tagHour.fillna(0)

In [ ]:
tagHour=tagHour.drop(['tags'],axis=1)

In [ ]:
tagHour=tagHour.fillna(0)

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
tagHourMatrix1 = tagHour.as_matrix()

In [ ]:
tagHourMatrix1.shape

In [ ]:
# tagHourMatrix1

t=np.array(tagHourMatrix1, dtype=np.float)

In [ ]:
t.shape

In [ ]:
output=output.fillna(0)

In [ ]:
outputMatrix1= output.as_matrix()

In [ ]:
outputMatrix1.shape

In [ ]:
t1=np.array(outputMatrix1,dtype=np.float)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(t, t1, test_size=0.33, random_state=42)

In [ ]:
np.argwhere(np.isnan(X_train))


In [ ]:
np.argwhere(np.isnan(y_train))

In [ ]:
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB


In [ ]:
classifier = BinaryRelevance(GaussianNB())


In [ ]:
classifier.fit(X_train, y_train)


In [ ]:
predictions = classifier.predict(X_test)


In [ ]:
# from sklearn.metrics import accuracy_score

# accuracy_score(y_test,predictions)

import sklearn.metrics


print(sklearn.metrics.hamming_loss(y_test, predictions))



In [ ]:
# X_test

In [ ]:
predictions1=predictions.toarray()

In [ ]:
predictions1

In [ ]:
y_test

In [ ]:
# for i,i1 in zip(predictions1,y_test):
#     for j,j1 in zip(i.shape[0],i1):
#         print(i," ",j)
#         print("-------")
#         print(i1,j1)
# #         print("Actual y_test ",i1[j1])
# #         print("Predicted value ",i[j])

In [ ]:
#predictions1.shape[1]

totalShape = predictions1.shape[0]*predictions1.shape[1]

In [ ]:
totalShape

In [ ]:
count=0
totalCount=0
for ii in range(predictions1.shape[0]):
    for jj in range(predictions1.shape[1]):
        if(predictions1[ii][jj]== y_test[ii][jj]):
            count+=1
            if(count==predictions1.shape[1]):
                totalCount+=1
                
    count=0        
            
print(totalCount)            

In [ ]:
accuracy1 = count/totalShape

In [ ]:
print("Accuracy in BinaryRelevance ==> ",accuracy1*100)